In [14]:
from preprocesstext import preProcessText
import pandas as pd
import tensorflow as tf
import numpy as np
import ast
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import json


In [2]:
with open(r'C:\Users\Mrulay\OneDrive - University of Windsor\uWindsor\COMP 8700 - Intro to AI\Project\data\chatbot\cornell movie-dialogs corpus\movie_conversations.txt') as f1:
    convs = f1.readlines()
convs = [x.strip("\n") for x in convs]
convs = [x.split(' +++$+++ ')[-1] for x in convs]

In [3]:
with open(r'C:\Users\Mrulay\OneDrive - University of Windsor\uWindsor\COMP 8700 - Intro to AI\Project\data\chatbot\cornell movie-dialogs corpus\movie_lines.txt') as f2:
    lines = f2.readlines()
lines = [x.strip("\n") for x in lines]
lines = [x.split(' +++$+++ ')[::len(x.split(' +++$+++ '))-1] for x in lines]

In [4]:
convs = [ast.literal_eval(x) for x in convs]
convs

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366'],
 ['L367', 'L368'],
 ['L401', 'L402', 'L403'],
 ['L404', 'L405', 'L406', 'L407'],
 ['L575', 'L576'],
 ['L577', 'L578'],
 ['L662', 'L663'],
 ['L693', 'L694', 'L695'],
 ['L696', 'L697', 'L698', 'L699'],
 ['L860', 'L861'],
 ['L862', 'L863', 'L864', 'L865'],
 ['L866', 'L867', 'L868', 'L869'],
 ['L870', 'L871', 'L872'],
 ['L924', 'L925'],
 ['L984', 'L985'],
 ['L1044', 'L1045'],
 ['L49', 'L50', 'L51'],
 ['L571', 'L572', 'L573'],
 ['L579', 'L580'],
 ['L595', 'L596', 'L597'],
 ['L598', 'L599', 'L600'],
 ['L659', 'L660'],
 ['L952', 'L953'],
 ['L394', 'L395'],
 ['L396', 'L397'],
 ['L589', 'L590', 'L591'],
 ['L592', 'L593'],
 ['L756', 'L757', 'L758'],
 ['L759', 'L760'],
 ['L164', 'L165'],
 ['L319', 'L320'],
 ['L441', 'L442', 'L443', 'L444', 'L445']

In [5]:
lines_to_preprocess = [x[-1] for x in lines]
lineIDs =  [x[0] for x in lines]
processed_lines = []
for line in lines_to_preprocess:
    processed_lines.append(preProcessText(line))
processed_lines

['they do not',
 'they do to',
 'i hope so',
 'she okay',
 'let us go',
 'wow',
 'okay  you are going to need to learn how to lie',
 'no',
 'i am kidding  you know how sometimes you just become this persona  and you do not know how to quit',
 'like my fear of wearing pastels',
 'the real you',
 'what good stuff',
 'i figured you would get to the good stuff eventually',
 'thank god  if i had to hear one more story about your coiffure',
 'me  this endless blonde babble i am like boring myself',
 'what crap',
 'do you listen to this crap',
 'no',
 'then guillermo says if you go any lighter you are going to look like an extra on 90210',
 'you always been this selfish',
 'but',
 'then that is all you had to say',
 'well no',
 'you never wanted to go out with me did you',
 'i was',
 'i looked for you back at the party but you always seemed to be occupied',
 'tons',
 'have fun tonight',
 'i believe we share an art instructor',
 'you know chastity',
 'looks like things worked out tonight huh',

In [6]:
lineDict = dict(zip(lineIDs,processed_lines))
lineDict

{'L1045': 'they do not',
 'L1044': 'they do to',
 'L985': 'i hope so',
 'L984': 'she okay',
 'L925': 'let us go',
 'L924': 'wow',
 'L872': 'okay  you are going to need to learn how to lie',
 'L871': 'no',
 'L870': 'i am kidding  you know how sometimes you just become this persona  and you do not know how to quit',
 'L869': 'like my fear of wearing pastels',
 'L868': 'the real you',
 'L867': 'what good stuff',
 'L866': 'i figured you would get to the good stuff eventually',
 'L865': 'thank god  if i had to hear one more story about your coiffure',
 'L864': 'me  this endless blonde babble i am like boring myself',
 'L863': 'what crap',
 'L862': 'do you listen to this crap',
 'L861': 'no',
 'L860': 'then guillermo says if you go any lighter you are going to look like an extra on 90210',
 'L699': 'you always been this selfish',
 'L698': 'but',
 'L697': 'then that is all you had to say',
 'L696': 'well no',
 'L695': 'you never wanted to go out with me did you',
 'L694': 'i was',
 'L693': 'i

In [7]:
for conv in convs:
    for i,id in enumerate(conv):
        conv[i] = lineDict[id]
convs
        

[['can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again',
  'well i thought we would start with pronunciation if that is okay with you',
  'not the hacking and gagging and spitting part  please',
  'okay then how bout we try out some french cuisine  saturday  night'],
 ['you are asking me out  that is so cute what is your name again',
  'forget it'],
 ['no no it is my fault  we did not have a proper introduction ',
  'cameron',
  'the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i cannot date until she does',
  'seems like she could get a date easy enough'],
 ['why',
  'unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something',
  'that is a shame'],
 ['gosh if only we could find kat a boyfriend', 'let me see what i can do'],
 ['cesc ma tete this is my head',
  'right  see  you are ready for the

In [8]:
ques, ans = [], []
for conv in convs:
    if len(conv)%2 != 0:
        conv.remove(conv[-1])

for conv in convs:
    for i in range (0, len(conv)):
        if i%2 == 0:
            ques.append(conv[i])
        elif i%2 != 0:
            ans.append(conv[i]) 
            

In [9]:
ques = ques[:30000]
ans = ans[:30000]

In [10]:
wordcount = dict()

for line in ques:
    for word in line.split():
        if word not in wordcount:
            wordcount[word] = 1
        else: 
            wordcount[word] += 1

for line in ans:
    for word in line.split():
        if word not in wordcount:
            wordcount[word] = 1
        else: 
            wordcount[word] += 1

In [11]:
t = 5
vocab = {}
word_num = 0
for word, count in wordcount.items():
    if count >= t:
        vocab[word] = word_num
        word_num += 1
vocab

{'can': 0,
 'we': 1,
 'make': 2,
 'this': 3,
 'quick': 4,
 'and': 5,
 'andrew': 6,
 'barrett': 7,
 'are': 8,
 'having': 9,
 'an': 10,
 'incredibly': 11,
 'public': 12,
 'break': 13,
 'up': 14,
 'on': 15,
 'the': 16,
 'again': 17,
 'not': 18,
 'hacking': 19,
 'part': 20,
 'please': 21,
 'you': 22,
 'asking': 23,
 'me': 24,
 'out': 25,
 'that': 26,
 'is': 27,
 'so': 28,
 'cute': 29,
 'what': 30,
 'your': 31,
 'name': 32,
 'no': 33,
 'it': 34,
 'my': 35,
 'fault': 36,
 'did': 37,
 'have': 38,
 'a': 39,
 'proper': 40,
 'thing': 41,
 'cameron': 42,
 'i': 43,
 'am': 44,
 'at': 45,
 'mercy': 46,
 'of': 47,
 'particularly': 48,
 'hideous': 49,
 'loser': 50,
 'sister': 51,
 'cannot': 52,
 'date': 53,
 'until': 54,
 'she': 55,
 'does': 56,
 'why': 57,
 'gosh': 58,
 'if': 59,
 'only': 60,
 'could': 61,
 'find': 62,
 'kat': 63,
 'boyfriend': 64,
 'ma': 65,
 'head': 66,
 'do': 67,
 'want': 68,
 'to': 69,
 'know': 70,
 'how': 71,
 'say': 72,
 'though': 73,
 'useful': 74,
 'things': 75,
 'like': 76,


In [12]:
for i in range(len(ans)):
    ans[i] = '<SOS> ' + ans[i] + ' <EOS>'

tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
x = len(vocab)
for token in tokens:
    vocab[token] = x
    x += 1

In [13]:
vocab['cameron'] = vocab['<PAD>']
vocab['<PAD>'] = 0

inv_vocab = {w:v for v, w in vocab.items()}

In [16]:
with open('vocab.json', 'w') as fp:
    json.dump(vocab, fp,  indent=4)

In [14]:
ids = []
for i,line in enumerate(ques):
    x = len(line.split())
    if x > 15:
        ques.remove(line)
        ids.append(i)
for id in ids:
    del ans[id]

In [15]:
tfidf = TfidfVectorizer()
response = tfidf.fit(processed_lines)
idf = tfidf.idf_
tfidf_values = (dict(zip(tfidf.get_feature_names_out(), idf)))
print (len(tfidf_values))

64894


In [16]:
encoder_inp = []
for line in ques:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])
        
    encoder_inp.append(lst)

In [17]:
decoder_inp = []
for line in ans:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])        
    decoder_inp.append(lst)

In [18]:
encoder_inp = pad_sequences(encoder_inp, 18, padding='post', truncating='post')
decoder_inp = pad_sequences(decoder_inp, 18, padding='post', truncating='post')

In [19]:
decoder_final_output = []
for i in decoder_inp:
    decoder_final_output.append(i[1:]) 

decoder_final_output = pad_sequences(decoder_final_output, 18, padding='post', truncating='post')

In [20]:
decoder_final_output = to_categorical(decoder_final_output, len(vocab))

In [21]:
VOCAB_SIZE = len(vocab)

In [22]:
encoder_inputs = Input(shape=(18,))
encoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True )(encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [state_h , state_c]

decoder_inputs = Input(shape=(18,))
decoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True)(decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(VOCAB_SIZE , activation=tf.keras.activations.softmax) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 18)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 18)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 18, 200)      1241600     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 18, 200)      1241600     ['input_2[0][0]']                
                                                                                              

In [23]:
checkpoint = ModelCheckpoint("best_model.h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto', period=1)

In [24]:
es = EarlyStopping(
    monitor='loss',
    patience=3,
    min_delta = 0.01,
    mode='min'
)

In [38]:
model.compile(loss='categorical_crossentropy',metrics=['acc'],optimizer='adam')

model.fit([encoder_inp, decoder_inp],decoder_final_output,epochs=20, callbacks=[es, checkpoint])

Epoch 1/20
785/785 [==============================] - ETA: 0s - loss: 0.3554 - acc: 0.8503
Epoch 1: loss did not improve from 0.35526
785/785 [==============================] - 93s 107ms/step - loss: 0.3554 - acc: 0.8503
Epoch 2/20
785/785 [==============================] - ETA: 0s - loss: 0.3417 - acc: 0.8581
Epoch 2: loss improved from 0.35526 to 0.34165, saving model to best_model.h5
785/785 [==============================] - 86s 109ms/step - loss: 0.3417 - acc: 0.8581
Epoch 3/20
785/785 [==============================] - ETA: 0s - loss: 0.3378 - acc: 0.8590
Epoch 3: loss improved from 0.34165 to 0.33779, saving model to best_model.h5
785/785 [==============================] - 87s 110ms/step - loss: 0.3378 - acc: 0.8590
Epoch 4/20
785/785 [==============================] - ETA: 0s - loss: 0.3360 - acc: 0.8588
Epoch 4: loss improved from 0.33779 to 0.33603, saving model to best_model.h5
785/785 [==============================] - 88s 112ms/step - loss: 0.3360 - acc: 0.8588
Epoch 5/20


In [35]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=(200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [42]:
for layer in model.layers:
    weights = layer.get_weights()
    if weights != []:
        np.savez(f'{layer.name}.npz', weights)                              

C:\Users\Mrulay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\lib\npyio.py:713: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


In [39]:
w_encoder_embeddings = np.load('embedding.npz', allow_pickle=True)
w_decoder_embeddings = np.load('embedding_1.npz', allow_pickle=True)
w_encoder_lstm = np.load('lstm.npz', allow_pickle=True)
w_decoder_lstm = np.load('lstm_1.npz', allow_pickle=True)
w_dense = np.load('dense.npz', allow_pickle=True)

In [29]:
encoder_inputs = Input(shape=(18,))
encoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True )(encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [state_h , state_c]

decoder_inputs = Input(shape=(18,))
decoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True)(decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(VOCAB_SIZE , activation=tf.keras.activations.softmax) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output)

In [30]:
model.layers[2].set_weights(w_encoder_embeddings['arr_0'])
model.layers[3].set_weights(w_decoder_embeddings['arr_0'])
model.layers[4].set_weights(w_encoder_lstm['arr_0'])
model.layers[5].set_weights(w_decoder_lstm['arr_0'])
model.layers[6].set_weights(w_dense['arr_0'])

In [31]:
enc_model, dec_model = make_inference_models()

In [36]:
def str_to_tokens(sentence:str):
    words = preProcessText(sentence)
    tokens_list = list()
    for word in words:
        try:
            tokens_list.append(vocab[word]) 
        except:
            tokens_list.append(vocab['<OUT>'])
    return pad_sequences([tokens_list], 18, padding='post', truncating='post')

In [43]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = vocab['<SOS>']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = inv_vocab[sampled_word_index] + ' '
        
        if sampled_word == 'end' or len(decoded_translation.split())>18:
            stop_condition = True
        if sampled_word != '<EOS> ':
            decoded_translation += sampled_word  
        if sampled_word == '<EOS> ' or len(decoded_translation.split()) > 13:
            stop_condition = True 
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print(decoded_translation)


yeah you cannot blame you this is a <OUT> piece of dining on <OUT> 
but is not everything the problem 
fuck procedure 
huh standing older son of <OUT> 


KeyboardInterrupt: 

In [4]:
import openpyxl
from nltk.corpus import stopwords
import nltk
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import pickle
stemmer = SnowballStemmer("english")
with open(r'C:\Users\Mrulay\OneDrive - University of Windsor\uWindsor\COMP 8700 - Intro to AI\Project\models\emotionDetection\linearSVC.pkl', 'rb') as fid:
    emotion_detector = pickle.load(fid)

In [5]:
te = "hello world, fuck all of you"

stop = stopwords.words('english')

def preProcessText(text):
    text = text.lower()
    text = text.replace(r'[^\w\s]+', '') 
    text = text.replace('@', '')
    text = ' '.join(word.lower() for word in text.split() if word not in stop)
    return text

tx = preProcessText(te)

In [6]:
def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

stx = stem_sentences(tx)